# Available spectra sources

This notebook shows the built-in source classes and a starter mapping you can
use with `bulk_download`.

In [1]:
import sys
import os
import pandas as pd
# Add the parent directory of the project to sys.path to ensure spectra_download is importable
notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
sys.path.append(os.path.dirname(notebook_dir))

In [2]:
from spectra_download import (
    ElodieSource,
    EsoHarpsSource,
    EsoNirpsSource,
    EsoUvesSource,
    NarvalSource,
)

available_sources = {
    "elodie": ElodieSource,
    "eso_harps": EsoHarpsSource,
    "eso_nirps": EsoNirpsSource,
    "eso_uves": EsoUvesSource,
    "narval": NarvalSource,
}

In [3]:
sources = {
    "elodie": ElodieSource(timeout=20, max_retries=3),
    "eso_harps": EsoHarpsSource(timeout=20, max_retries=3),
    "eso_nirps": EsoNirpsSource(timeout=20, max_retries=3),
    "eso_uves": EsoUvesSource(timeout=20, max_retries=3),
    "narval": NarvalSource(timeout=20, max_retries=3),
}

In [4]:
fgk_stars = pd.read_csv('../data/fgk.txt', delimiter='\t')
fgk_stars.head()

,star,group,ID,ID_alt,star_alt1,star_alt2,origin,snr,R,Rmax,...,e[Ti 1/H],n[Ti 1/H],[Ti 2/H],e[Ti 2/H],n[Ti 2/H],[V 1/H],e[V 1/H],n[V 1/H],n_spectra,flag
0,HIP101345,G Subgiant (IV),HD195564_HAR_1,HIP101345_HARPS_1,HD195564,-,HARPS,902,42000,115000,...,0.023,34,0.062,0.027,7,-0.121,0.024,27,2,-
1,HIP101345,G Subgiant (IV),HD195564_NAR_1,HIP101345_NARVAL_1,HD195564,-,NARVAL,369,42000,68000,...,0.023,34,0.062,0.027,7,-0.121,0.024,27,2,-
2,HIP10234,K Giant (III),HD13468_FER_1,HIP10234_FEROS_1,HD13468,-,FEROS,121,42000,48000,...,0.059,36,-0.221,0.042,8,-0.532,0.061,29,2,-
3,HIP10234,K Giant (III),HD13468_HAR_1,HIP10234_HARPS_1,HD13468,-,HARPS,95,42000,115000,...,0.059,36,-0.221,0.042,8,-0.532,0.061,29,2,-
4,HIP102422,K Subgiant (IV),HD198149_NAR_1,HIP102422_NARVAL_1,HD198149,-,NARVAL,908,42000,68000,...,0.062,35,-0.218,0.031,8,-0.447,0.047,30,2,-


In [5]:
fgk_stars[fgk_stars.origin=='ELODIE'].head()

,star,group,ID,ID_alt,star_alt1,star_alt2,origin,snr,R,Rmax,...,e[Ti 1/H],n[Ti 1/H],[Ti 2/H],e[Ti 2/H],n[Ti 2/H],[V 1/H],e[V 1/H],n[V 1/H],n_spectra,flag
28,HIP112731,K Giant (III),HD216174_ELO_1,HIP112731_ELODIE_1,HD216174,-,ELODIE,204,42000,45000,...,0.108,28,-0.297,0.037,8,-0.601,0.099,27,2,-
39,HIP114971,K Giant (III),HD219615_ELO_1,HIP114971_ELODIE_1,HD219615,-,ELODIE,143,42000,45000,...,0.057,38,-0.233,0.034,8,-0.312,0.043,30,2,-
235,HIP57939,G Dwarf (V),HD103095_ELO_1,HIP57939_ELODIE_1,HD103095,Gmb1830,ELODIE,359,42000,45000,...,0.090,31,-1.024,0.078,8,-0.921,0.040,29,2,-
266,HIP69673,K Giant (III),HD124897_ELO_1,HIP69673_ELODIE_1,HD124897,Arcturus,ELODIE,902,42000,45000,...,0.070,53,-0.332,0.013,19,-0.445,0.072,57,5,-
343,HIP88348,G Dwarf (V),HD164922_ELO_1,HIP88348_ELODIE_1,HD164922,-,ELODIE,655,42000,45000,...,0.032,51,0.162,0.039,12,0.276,0.036,44,3,-


In [6]:
src = sources["elodie"]
records = src.download("HD190007", raw_save_path="../data/spectra/elodie/", zarr_paths="../data/elodie.zarr")

Download attempt failed
Download attempt failed


In [8]:
from astropy.io import fits

# Attempt to open the FITS file as requested
spec = fits.open('../data/elodie_19940917_0011.fits')

In [9]:
spec.info()

Filename: ../data/elodie_19940917_0011.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  INTENSITY     1 PrimaryHDU     113   (56000,)   float32   
  1  FCANOR        1 BinTableHDU     25   147R x 1C   [1E]   
  2  NOISE         1 ImageHDU       110   (56000,)   float32   


In [10]:
import numpy as np

In [11]:
import astropy.units as u

hdr = spec[0].header

n = hdr["NAXIS1"]
crval = hdr["CRVAL1"]      # in units of 0.1 nm
cdelt = hdr["CDELT1"]
crpix = hdr["CRPIX1"]

wave = (crval + (np.arange(n) + 1 - crpix) * cdelt) * u.AA

In [12]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [13]:
import zarr

# Open the Zarr store for Elodie data
spectra = zarr.open_group('../data/elodie.zarr')

In [14]:
spectra.tree()

/
└── spectra
    └── HD190007
        ├── ccf (12, 141) float64
        ├── intensity (14, 56000) float64
        ├── n_ccf_points (12,) int32
        ├── n_points (14,) int32
        └── wavelength (14, 56000) float64